In [1]:
import scrapy
import re
import json
import logging
import pandas as pd
import selenium
import time
import sys
import linecache
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
import pymongo
import datetime
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from scrapy.crawler import CrawlerProcess
from time import sleep
from selenium.webdriver.common.keys import Keys
from time import sleep 
from pymongo import MongoClient 
from selenium.webdriver.common.by import By

# LAZADA Scraping

In [2]:
# Print exception
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print ('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

## Top 400 per category

In [3]:
# Scraping the page
def scrape_detail_page(driver,category,collection):
    time.sleep(5)
    # scraping page
    path = driver.find_elements_by_class_name('c2prKC')
    check = driver.find_elements_by_class_name('c1DXz4')[0].text
    while(check is None):
        path = driver.find_elements_by_class_name('c2prKC')
        time.sleep(2)
    result = pd.DataFrame()
    # print(len(path))
    for x in range(len(path)):
        error = 0
        while(error<5):
            try:
                location = path[x].location['y']-100
                driver.execute_script("window.scrollTo(0, {});".format(location))
                try:
                    origin_price = re.sub("\.","",re.sub("Rp","",path[x].find_element_by_css_selector('.c3lr34').find_element_by_css_selector('.c13VH6').text))
                except:
                    origin_price = re.sub("\.","",re.sub("Rp","",path[x].find_element_by_css_selector('.c3gUW0').find_element_by_css_selector('.c13VH6').text))

                try:
                    discounted_price = re.sub("\.","",re.sub("Rp","",path[x].find_element_by_css_selector('.c3gUW0').find_element_by_css_selector('.c13VH6').text))
                except:
                    discounted_price = origin_price

                image = path[x].find_element_by_xpath('div/div/div[1]/div[1]/a/img').get_attribute('src')
                sku = path[x].get_attribute('data-item-id')
                name = driver.find_elements_by_class_name('c16H9d')[x].text

                while(image is None):
                    location = path[x].location['y']-100
                    driver.execute_script("window.scrollTo(0, {});".format(location))
                    image = path[x].find_element_by_xpath('div/div/div[1]/div[1]/a/img').get_attribute('src')
                    time.sleep(1)

                # Return result
                date = datetime.datetime.now().strftime("%Y-%m-%d")
                new_result = pd.DataFrame([{'sku':sku,'name':name, 'origin_price':origin_price, 'discounted_price':discounted_price, 'image':image,'category':category,'scraping_date':date}])
                result = result.append(new_result)
                #print(new_result)
                
                break
                
            except:
                PrintException()
                error=error+1
            
    # save to mongodb
    try:
        collection.insert_many(result.to_dict('records'))
    except:
        PrintException()
        pass

    #driver.close()
    return result

In [15]:
def scrape_url2(url,category,collection):
        
    # Connect to mongodb
    try: 
        conn = MongoClient() 
        print("Connected successfully!!!") 
    except:   
        print("Could not connect to MongoDB") 
    # database
    db = conn.lazada
    # Created or Switched to collection names: pakaian_wanita 
    collection = db[collection]
    
    # Load WebDriver and navigate to the page url.
    # This will open a browser window.
    url1 = url + "?page=1"
    driver = webdriver.Firefox()
    driver.get(url1)

    
    # Start Scraping
    result = pd.DataFrame()
    page = 1
    start = 0
    image_unique=0
    while(image_unique<400):
        # Go to catalogue page
        result = result.append(scrape_detail_page(driver,category,collection))
        result = result.drop_duplicates(['image'])
        image_unique=len(result)

        # Go to the next page
        try:
            # check_next = driver.find_elements_by_xpath('/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[3]/div/ul/li[9]/a')[0]
            check_next = driver.find_element_by_css_selector('.ant-pagination-next').find_element_by_xpath('a')
            try:
                check_next.click()
            except:
                time.sleep(10)
                PrintException()
                driver.refresh()
                #en =  driver.find_element_by_css_selector(".nc_bg")
                #move = ActionChains(driver)
                #move.click_and_hold(en).move_by_offset(100, 0).release().perform()
                #time.sleep(5)
            page = page+1
            next_page = url + "?page=" + str(page)
            print("Go to next page: " + next_page)
            #driver.close()
            #driver = webdriver.Firefox()
            #driver.get(next_page)
        except:
            PrintException()
            break
    
    #driver.close()
    return result


In [16]:
# MAIN
lazada_category = pd.read_csv('C:/Users/tamu/SharePoint/Product Development - Documents/Merchandising Analytics/Analysis - QBR Jul19/Data/scraping/lazada_category.csv',sep=',')
result = pd.DataFrame()
for x in range(0,1):
    category = lazada_category.iloc[x]
    print(category['category'])
    
    url = category['url']
    collection = category['collection']+"_top"
    
    result = scrape_url2(url,category['category'],collection)
    print(result.head())

pakaian_wanita > dress
Connected successfully!!!
Go to next page: https://www.lazada.co.id/gaun-wanita/?page=2
EXCEPTION IN (<ipython-input-3-c28bbe36bb3f>, LINE 28 "image = path[x].find_element_by_xpath('div/div/div[1]/div[1]/a/img').get_attribute('src')"): Message: Unable to locate element: div/div/div[1]/div[1]/a/img

EXCEPTION IN (<ipython-input-15-76eb3453a701>, LINE 37 "check_next.click()"): Message: Element <a class="ant-pagination-item-link"> is not clickable at point (1260,581) because another element <div class="sufei-dialog-mask"> obscures it

Go to next page: https://www.lazada.co.id/gaun-wanita/?page=3


IndexError: list index out of range

In [15]:
en =  driver.find_element_by_css_selector(".nc_iconfont")

NameError: name 'driver' is not defined

## By Brand

In [3]:
# Scraping the page
def scrape_detail_page2(driver,category,collection,brand):
    time.sleep(5)
    # scraping page
    path = driver.find_elements_by_class_name('c2prKC')
    check = driver.find_elements_by_class_name('c1DXz4')[0].text
    while(check is None):
        path = driver.find_elements_by_class_name('c2prKC')
        time.sleep(2)
    result = pd.DataFrame()

    for x in range(len(path)):
        error = 0
        while(error<5):
            try:
                location = path[x].location['y']-100
                driver.execute_script("window.scrollTo(0, {});".format(location))
                try:
                    origin_price = re.sub("\.","",re.sub("Rp","",path[x].find_element_by_css_selector('.c3lr34').find_element_by_css_selector('.c13VH6').text))
                except:
                    origin_price = re.sub("\.","",re.sub("Rp","",path[x].find_element_by_css_selector('.c3gUW0').find_element_by_css_selector('.c13VH6').text))

                try:
                    discounted_price = re.sub("\.","",re.sub("Rp","",path[x].find_element_by_css_selector('.c3gUW0').find_element_by_css_selector('.c13VH6').text))
                except:
                    discounted_price = origin_price

                image = path[x].find_element_by_xpath('div/div/div[1]/div[1]/a/img').get_attribute('src')
                sku = path[x].get_attribute('data-item-id')
                name = driver.find_elements_by_class_name('c16H9d')[x].text

                while(image is None):
                    location = path[x].location['y']-100
                    driver.execute_script("window.scrollTo(0, {});".format(location))
                    image = path[x].find_element_by_xpath('div/div/div[1]/div[1]/a/img').get_attribute('src')
                    time.sleep(1)

                # Return result
                date = datetime.datetime.now().strftime("%Y-%m-%d")
                new_result = pd.DataFrame([{'brand':brand,'sku':sku,'name':name, 'origin_price':origin_price, 'discounted_price':discounted_price, 'image':image,'category':category,'scraping_date':date}])
                result = result.append(new_result)

                
                break
                
            except:
                PrintException()
                error=error+1
            
    # save to mongodb
    try:
        collection.insert_many(result.to_dict('records'))
    except:
        PrintException()
        pass

    #driver.close()
    return result

In [4]:
def scrape_url3(url,category,collection,brand):
        
    # Connect to mongodb
    try: 
        conn = MongoClient() 
        print("Connected successfully!!!") 
    except:   
        print("Could not connect to MongoDB") 
    # database
    db = conn.lazada_brand
    # Created or Switched to collection names: pakaian_wanita 
    collection = db[collection]
    
    # Load WebDriver and navigate to the page url.
    # This will open a browser window.
    url1 = url + "&page=1"
    driver = webdriver.Firefox()
    driver.get(url1)

    
    # Start Scraping
    result = pd.DataFrame()
    page = 1
    start = 0
    image_unique=0
    check = "false"
    while(image_unique<400 and check == "false"):
        # Go to catalogue page
        result = result.append(scrape_detail_page2(driver,category,collection,brand))
        # Go to the next page
        try:
            result = result.drop_duplicates(['image'])
            image_unique=len(result)
            check_next = driver.find_element_by_css_selector('.ant-pagination-next').find_element_by_xpath('a')
            check = driver.find_element_by_css_selector('.ant-pagination-next').get_attribute('aria-disabled')
            check_next.click()
            page = page+1
            next_page = url + "&page=" + str(page)
            print("Go to next page: " + next_page)
        except:
            PrintException()
            break
    
    driver.close()
    return result


In [9]:
# MAIN
lazada_category = pd.read_csv('C:/Users/tamu/SharePoint/Product Development - Documents/Merchandising Analytics/Analysis - QBR Jul19/Data/scraping/lazada_category.csv',sep=',')
result = pd.DataFrame()
for x in range(1,len(result)):
    category = lazada_category.iloc[x]
    print(category['category'])
    
    brand = '/?coolkids-official-store&from=wangpu'
    url = category['url']+brand
    collection = category['collection']+"_brand"
    
    result = scrape_url3(url,category['category'],collection,brand)
    print(result.head())

pakaian_anak > dress
Connected successfully!!!
Go to next page: https://www.lazada.co.id/fashion-dress-anak-perempuan/?coolkids-official-store&from=wangpu&page=2
                                   brand              category  \
0  /?coolkids-official-store&from=wangpu  pakaian_anak > dress   
0  /?coolkids-official-store&from=wangpu  pakaian_anak > dress   
0  /?coolkids-official-store&from=wangpu  pakaian_anak > dress   
0  /?coolkids-official-store&from=wangpu  pakaian_anak > dress   
0  /?coolkids-official-store&from=wangpu  pakaian_anak > dress   

  discounted_price                                              image  \
0           359920  https://id-test-11.slatic.net/p/1366739c4f220f...   
0           151920  https://id-live-01.slatic.net/original/64b4ee9...   
0           359920  https://id-test-11.slatic.net/p/293b18df91aaaf...   
0           143920  https://id-live-01.slatic.net/original/29cb823...   
0           219900  https://id-live-01.slatic.net/original/efa062a...   

  